<div class="alert alert-block alert-danger">
    <b> Unraveling the cytoskeletal architecture of cancer cells: a novel computational approach to predict cell fate - CODE</b> 
    
 **6 - FEATURE EXTRACTION**
    
**Author**: Diogo Fróis Vieira
</div>

<div class="alert alert-block alert-info">
    
 **IMPORTS**
    
</div>

In [8]:
# Math, image processing and other useful libraries
from __future__ import print_function, unicode_literals, absolute_import, division
import os
import sys
sys.path.append(os.path.abspath('..'))

import pandas as pd
import numpy as np
import cv2
from collections import OrderedDict
import copy
import math
import pickle
from matplotlib.ticker import MaxNLocator
from itertools import combinations

# Image processing
from skimage.measure import regionprops
from skimage.filters import meijering, sato, frangi, hessian, threshold_otsu
from skimage.morphology import extrema, skeletonize
from skimage.transform import probabilistic_hough_line
from skimage.draw import disk, circle_perimeter
from scipy.ndimage import gaussian_filter, grey_closing
from scipy.spatial import distance_matrix
from skimage import data, restoration, util
from roipoly import RoiPoly
from matplotlib_scalebar.scalebar import ScaleBar
from biosppy.signals import tools
from biosppy.stats import pearson_correlation
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Plotting
import matplotlib.pyplot as plt
import matplotlib.cm as pltc
import matplotlib.colors as colors
import seaborn as sns

# Widgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

# Feature Extraction (.py files by Teresa Parreira)
# from CytoSkeletonPropsMorph import CytoSkeletonPropsMorph
# from CytoSkeletonRegionPropsInt import RegionPropsInt
# from FreqAnalysis import FreqAnalysis
# from GLCM import GLCM

# Graph
import sknw
import networkx as nx
from scipy.signal import argrelextrema

# 
from skan import Skeleton, summarize,draw
from skan.csr import skeleton_to_csgraph, sholl_analysis,make_degree_image
import scipy as sp
import scipy.sparse
from matplotlib.patches import Circle

from packageCYSK.utils import cv2toski,pylsdtoski,polar_to_cartesian, remove_not1D, quantitative_analysis,hist_bin,hist_lim,branch,graphAnalysis
from packageCYSK.importing import *
from packageCYSK.preprocessingCYTO import *
#from framework.PreProcessingNUCL import excludeborder, nuclei_preprocessing, df_nuclei_preprocessing, nuclei_segmentation
from packageCYSK.processing import *
from packageCYSK.visualization import *
#from packageCYSK.analysis import plot_barplot
from packageCYSK.processing_LSF import *
from packageCYSK.processing_DCF import *
from packageCYSK.processing_CNF import *
from packageCYSK.feature_extraction import *
#from fractal_dimension import fractal_dimension
#from fractal_analysis_fxns import boxcount,boxcount_grayscale,fractal_dimension,fractal_dimension_grayscale,fractal_dimension_grayscale_DBC

print('📚 All libraries successfully imported 📚')

📚 All libraries successfully imported 📚


<div class="alert alert-block alert-info">
    
 **DIRECTORIES**
    
</div>

In [14]:
# Define dataset folder.
folder      = os.path.dirname(os.getcwd()) + "\\Datasets\\Set 1-a-tubulin_Sofia"

# Import images from CYTO, NUCL and RGB folders.
options     = ["CYTO","NUCL","RGB"]

# Function to automatically retrieve image labels and indexes.
denominator = label_tubulin

# Dictionary with keys "CYTO", "NUCL", "RGB".
data = init_import(folder,options,denominator)

# Import ROIs
ROIs = pickle.load(open(folder + '//ROIs//ROIs_TEST.pkl', 'rb'))

# Import processed results
ResultsDF = pd.read_pickle(folder + "\\ResultsDF_TEST.pkl")

TiffPage 0: IndexError: tuple index out of range


>>> [RGB] added.
>>> [CYTO] added.
>>> [NUCL] added.


In [15]:
ResultsDF

,Path,Name,Img Index,Label,Image Size,Resolution,Mask,Skeleton,Patch:Deconvoluted Cyto,Patch:Deconvoluted Nucl,Patch:Skeleton Max,Offset,Nucleus Contour,Nucleus Centroid,Lines
0,C:\Users\diogo\OneDrive - Universidade de Lisb...,MAX_9_6IF_CHO_L_w3z_decon_ch02_PS.tif,9,WT,"(1040, 1388)","(1, 1.6e-05, 1.6e-05)","([676, 676, 676, 676, 677, 677, 677, 677, 677,...","([688, 688, 688, 688, 689, 689, 690, 690, 690,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[676, 676, 676, 676, 677, 677, 677, 677, 677,...","[[677, 686], [676, 687], [675, 687], [674, 688...","(711.042, 677.249)","[((668.185, 687.963), (669.815, 692.037)), ((6..."


<div class="alert alert-block alert-info">
    
 **SELECT FEATURES**
    
</div>

In [4]:
# CREATE FEATURE LIST:
import ipywidgets as widgets

# Define the groups of features
groups = {
    'DCF': [
        'DCF:Area',
        'DCF:BB Area',
        'DCF:Perimeter',
        'DCF:Area convex',
        'DCF:Centroid',
        'DCF:Weighted Centroid',
        'DCF:Centroid Divergence',
        'DCF:Equivalent Diameter',
        'DCF:Extent',
        'DCF:Major Axis Length',
        'DCF:Minor Axis Length',
        'DCF:Height',
        'DCF:Euler Number',
        'DCF:Eccentricity',
        'DCF:Circularity',
        'DCF:Roundness',
        'DCF:Orientation',
        'DCF:Solidity',
        'DCF:Roughness',
        'DCF:Hu Moment #1',
        'DCF:Hu Moment #2',
        'DCF:Hu Moment #3',
        'DCF:Weighted Hu Moment #1',
        'DCF:Weighted Hu Moment #1',
        'DCF:Weighted Hu Moment #1',
        'DCF:Feret Diameter Max',
        'DCF:Crofton Perimeter',

        'DCF:Mean Intensity',
        'DCF:Std',
        'DCF:Variance',
        'DCF:Skewness',
        'DCF:Kurtosis',
        'DCF:Contrast',
        'DCF:Max Intensity',
        'DCF:Min Intensity',
        'DCF:Entropy',
        'DCF:Inertia Tensor Highest Eigenvalue',
        'DCF:Inertia Tensor Lowest Eigenvalue',

        'DCF:Uniformity',
        'DCF:Invariant Uniformity',
        'DCF:GLCM Entropy',
        'DCF:GLCM Invariant Entropy',
        'DCF:Correlation',
        'DCF:Invariant Correlation',
        'DCF:Dissimilarity',
        'DCF:Invariant Dissimilarity',
        'DCF:Contrast',
        'DCF:Invariant Contrast',
        'DCF:Homogeneity',
        'DCF:Invariant Homogeneity',
        'DCF:Energy',
        'DCF:Invariant Energy',

        'DCF:Mean Gabor Power',
        'DCF:Gabor Variance',
        'DCF:Gabor Mean',
        'DCF:Gabor Energy',
        'DCF:Gabor Entropy',
        'DCF:Mean Spectral Magnitude',
        'DCF:Mean Spectral Power'
    ],
    'DNF': [
            'DNF:Area',
            'DNF:BB Area',
            'DNF:Perimeter',
            'DNF:Area convex',
            'DNF:Centroid',
            'DNF:Weighted Centroid',
            'DNF:Centroid Divergence',
            'DNF:Equivalent Diameter',
            'DNF:Extent',
            'DNF:Major Axis Length',
            'DNF:Minor Axis Length',
            'DNF:Height',
            'DNF:Euler Number',
            'DNF:Eccentricity',
            'DNF:Circularity',
            'DNF:Roundness',
            'DNF:Orientation',
            'DNF:Solidity',
            'DNF:Roughness',
            'DNF:Hu Moment #1',
            'DNF:Hu Moment #2',
            'DNF:Hu Moment #3',
            'DNF:Weighted Hu Moment #1',
            'DNF:Weighted Hu Moment #1',
            'DNF:Weighted Hu Moment #1',
            'DNF:Feret Diameter Max',
            'DNF:Crofton Perimeter',
            
            'DNF:Mean Intensity',
            'DNF:Std',
            'DNF:Variance',
            'DNF:Skewness',
            'DNF:Kurtosis',
            'DNF:Contrast',
            'DNF:Max Intensity',
            'DNF:Min Intensity',
            'DNF:Entropy',
            'DNF:Inertia Tensor Highest Eigenvalue',
            'DNF:Inertia Tensor Lowest Eigenvalue',
            
            'DNF:Uniformity',
            'DNF:Invariant Uniformity',
            'DNF:GLCM Entropy',
            'DNF:GLCM Invariant Entropy',
            'DNF:Correlation',
            'DNF:Invariant Correlation',
            'DNF:Dissimilarity',
            'DNF:Invariant Dissimilarity',
            'DNF:Contrast',
            'DNF:Invariant Contrast',
            'DNF:Homogeneity',
            'DNF:Invariant Homogeneity',
            'DNF:Energy',
            'DNF:Invariant Energy',
            
            'DNF:Mean Gabor Power',
            'DNF:Gabor Variance',
            'DNF:Gabor Mean',
            'DNF:Gabor Energy',
            'DNF:Gabor Entropy',
            'DNF:Mean Spectral Magnitude',
            'DNF:Mean Spectral Power'
    ],
    'LSF': [
            'LSF2D:Angles',
            'LSF2D:Distances to Centroid',
            'LSF2D:Triangle Areas',
            'LSF2D:Line Lengths',
            'LSF2D:Theta',
            'LSF2D:Angle Difference',
            'LSF2D:Std. Angle Difference',
            'LSF2D:Local Line Distance',
            'LSF2D:Std. Local Line Distance',
            'LSF2D:PAD',
           
            'LSF1D:Number of Lines',
            #'LSF1D:Radial Score',
            'LSF1D:Complete Diameter Distance',
            'LSF1D:Average Diameter Distance',
            'LSF1D:TAD',
            'LSF1D:OOP',
            'LSF1D:HI',
            'LSF1D:MCM'
    ],
    'CNF': [
            'CNF1D:Number of Branches',

            'CNF2D:Branch Lengths',
        
            'CNF2D:Branch Orientations',
        
            'CNF2D:Branch Orientations PCA',
        
            'CNF2D:Local Average Branch Distances' ,
        
            'CNF2D:Local Average Bundling Score' ,
            
            'CNF2D:Local Average Branch Orientation',
        
            'CNF2D:Distances to Centroid',
            
            'CNF2D:mean-pixel-value',
            
            
            'CNF2D:stdev-pixel-value',
            
            
            'CNF2D:euclidean-distance',
            
            
            'CNF1D:Number of Endpoint-to-endpoint (isolated branch)',
            'CNF1D:Ratio of Endpoint-to-endpoint (isolated branch)',
            'CNF1D:Mean of Endpoint-to-endpoint (isolated branch) branch-distance',
            'CNF1D:Std of Endpoint-to-endpoint (isolated branch) branch-distance',
            'CNF1D:Mean of Endpoint-to-endpoint (isolated branch) mean-pixel-value',
            'CNF1D:Std of Endpoint-to-endpoint (isolated branch) mean-pixel-value',
            'CNF1D:Mean of Endpoint-to-endpoint (isolated branch) stdev-pixel-value',
            'CNF1D:Std of Endpoint-to-endpoint (isolated branch) stdev-pixel-value',
            'CNF1D:Mean of Endpoint-to-endpoint (isolated branch) euclidean-distance',
            'CNF1D:Std of Endpoint-to-endpoint (isolated branch) euclidean-distance',
            
            'CNF1D:Number of Junction-to-endpoints',
            'CNF1D:Ratio of Junction-to-endpoints',
            'CNF1D:Mean of Junction-to-endpoints branch-distance',
            'CNF1D:Std of Junction-to-endpoints branch-distance',
            'CNF1D:Mean of Junction-to-endpoints mean-pixel-value',
            'CNF1D:Std of Junction-to-endpoints mean-pixel-value',
            'CNF1D:Mean of Junction-to-endpoints stdev-pixel-value',
            'CNF1D:Std of Junction-to-endpoints stdev-pixel-value',
            'CNF1D:Mean of Junction-to-endpoints euclidean-distance',
            'CNF1D:Std of Junction-to-endpoints euclidean-distance',
            
            'CNF1D:Number of Junction-to-junctions',
            'CNF1D:Ratio of Junction-to-junctions',
            'CNF1D:Mean of Junction-to-junctions branch-distance',
            'CNF1D:Std of Junction-to-junctions branch-distance',
            'CNF1D:Mean of Junction-to-junctions mean-pixel-value',
            'CNF1D:Std of Junction-to-junctions mean-pixel-value',
            'CNF1D:Mean of Junction-to-junctions stdev-pixel-value',
            'CNF1D:Std of Junction-to-junctions stdev-pixel-value',
            'CNF1D:Mean of Junction-to-junctions euclidean-distance',
            'CNF1D:Std of Junction-to-junctions euclidean-distance',
            
            'CNF1D:Number of Isolated cycles',
            'CNF1D:Ratio of Isolated cycles',
            'CNF1D:Mean of Isolated cycles branch-distance',
            'CNF1D:Std of Isolated cycles branch-distance',
            'CNF1D:Mean of Isolated cycles mean-pixel-value',
            'CNF1D:Std of Isolated cycles mean-pixel-value',
            'CNF1D:Mean of Isolated cycles stdev-pixel-value',
            'CNF1D:Std of Isolated cycles stdev-pixel-value',
            'CNF1D:Mean of Isolated cycles euclidean-distance',
            'CNF1D:Std of Isolated cycles euclidean-distance',
            
            'CNF1D:Sholl Crossings Cyto mean',
            'CNF1D:Sholl Crossings Cyto median',
            'CNF1D:Sholl Crossings Cyto min',
            'CNF1D:Sholl Crossings Cyto max',
            'CNF1D:Sholl Crossings Cyto max_amp',
            'CNF1D:Sholl Crossings Cyto var',
            'CNF1D:Sholl Crossings Cyto std_dev',
            'CNF1D:Sholl Crossings Cyto abs_dev',
            'CNF1D:Sholl Crossings Cyto kurtosis',
            'CNF1D:Sholl Crossings Cyto skewness',
            
            'CNF1D:Sholl Crossings Nuclei mean',
            'CNF1D:Sholl Crossings Nuclei median',
            'CNF1D:Sholl Crossings Nuclei min',
            'CNF1D:Sholl Crossings Nuclei max',
            'CNF1D:Sholl Crossings Nuclei max_amp',
            'CNF1D:Sholl Crossings Nuclei var',
            'CNF1D:Sholl Crossings Nuclei std_dev',
            'CNF1D:Sholl Crossings Nuclei abs_dev',
            'CNF1D:Sholl Crossings Nuclei kurtosis',
            'CNF1D:Sholl Crossings Nuclei skewness'
    ]
}

# Create the checkboxes
checkboxes = {}
for group, features in groups.items():
    checkbox_group = []
    for feature in features:
        checkbox = widgets.Checkbox(value=True, description=feature)
        checkbox_group.append(checkbox)
    checkboxes[group] = checkbox_group

# Create the button
generate_button = widgets.Button(description='Generate List')
features = []

# Function to handle button click event
def generate_list(button):
    global features
    features = []
    for group in checkboxes.values():
        features.extend([checkbox.description for checkbox in group if checkbox.value])
    #print(f'Features: {features}')
    print('Variable "features" created.')

# Attach the function to the button click event
generate_button.on_click(generate_list)

# Create the checkbox groups and add vertical spacing
groups_layout = []
for group, checkboxes_group in checkboxes.items():
    checkboxes_group_layout = widgets.HBox(checkboxes_group)
    groups_layout.append(widgets.VBox([checkboxes_group_layout, widgets.HTML('<br>')]))

# Create the final layout
layout = widgets.VBox(groups_layout + [generate_button])

# Display the layout
layout


Variable "features" created.
Variable "features" created.


<div class="alert alert-block alert-info">
    
 **SINGLE CELL**
    
</div>

In [13]:
def dropdown_cell_eventhandler(change):
    global cell
    cell = dropdown_image.value
    lab = ResultsDF.loc[cell]['Label']
    print('Cell ' + str(cell) + ' (' + str(lab) +') selected.')

dropdown_image = widgets.Dropdown(description =  'Select Cell:' , options = ['...'] + list(ResultsDF.index))
dropdown_image.observe(dropdown_cell_eventhandler, names='value')
display(dropdown_image)

Dropdown(description='Select Cell:', options=('...', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,…

Cell 32 (Del38_46) selected.


In [6]:
ResultsDF = feature_extractor(ResultsDF.loc[cell],data,features)

TiffPage 0: IndexError: tuple index out of range
C:\Users\diogo\anaconda3\envs\Cytosk_recentpy\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 91, using nperseg = 91
  warnings.warn('nperseg = {0:d} is greater than input length '
TiffPage 0: IndexError: tuple index out of range


In [7]:
ResultsDF

,Path,Name,Img Index,Label,Image Size,Resolution,Mask,Skeleton,Patch:Deconvoluted Cyto,Patch:Deconvoluted Nucl,...,LSF1D:Number of Lines,LSF1D:OOP,CNF1D:Number of Branches,CNF2D:Branch Lengths,CNF2D:Branch Orientations,CNF2D:Branch Orientations PCA,CNF2D:Local Average Branch Distances,CNF2D:Local Average Bundling Score,CNF2D:Local Average Branch Orientation,CNF2D:Distances to Centroid
0,C:\Users\diogo\OneDrive - Universidade de Lisb...,MAX_9_6IF_CHO_L_w3z_decon_ch02_PS.tif,9,WT,"(1040, 1388)","(1, 1.6e-05, 1.6e-05)","([676, 676, 676, 676, 677, 677, 677, 677, 677,...","([688, 688, 688, 688, 689, 689, 690, 690, 690,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",...,142,0.014352,137,"[7.725483399593903e-05, 7.06274169979695e-05, ...","[67.5, 78.75, 154.6875, 6.923076923076923, 118...","[67.5, 72.85656151139553, 171.99931576466966, ...","[10.054, 8.234, 10.293, 7.551, 7.107, 10.506, ...","[766.949, 811.736, 468.526, 65.312, 672.193, 1...","[54.643, 78.75, 32.236, 6.923, 73.125, 9.643, ...","[7.084, 2.72, 5.771, 3.821, 5.981, 8.258, 8.52..."


<div class="alert alert-block alert-info">
    
 **DATAFRAME OF PROCESSED CELLS**
    
</div>

In [ ]:
ResultsDF = df_feature_extractor(ResultsDF = ResultsDF,
                                 listfeats = features)

<div class="alert alert-block alert-info">
    
 **SAVE RESULTS**
    
</div>

In [7]:
ResultsDF.to_pickle(folder + '//ResultsDF_TEST_DONE.pkl')   

---